# Seoul Bike Sharing

**Attribute Information:**

**-Rented Bike count:**  - Count of bikes rented at each hour, continuous numeric value<br>
**-Hour:** Hour of the day numeric value <br>
**-Temperature:** in Celsius numeric value <br>
**-Humidity:** in %, numeric value <br>
**-Windspeed:** in m/s, numeric value<br>
**-Visibility:** in 10m, numeric value<br>
**-Dew point temperature:** in Celsius, numeric value<br>
**-Solar radiation:** MJ/m2, numeric value<br>
**-Rainfall:** in mm, numeric value<br>
**-Snowfall:** in cm, numeric value<br>
**-Seasons:** Winter, Spring, Summer, Autumn categorical value<br>
**-Holiday:** Holiday/No holiday binary value<br>
**-Functional Day:** NoFunc(Non Functional Hours), Fun(Functional hours) binary value<br>


### Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from scipy import stats
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, Normalizer
from sklearn.neighbors import KNeighborsRegressor
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import metrics

import sys
sys.path.insert(1, '../RegressionAlgorithms/')
import linearRegression
from knn import *

### Get the Data

In [ ]:
data = pd.read_csv('SeoulBikeData.csv', delimiter = ',', engine='python')

In [ ]:
data

### Basic Data Information 

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
print(data.isnull().sum(axis=0))

### Exploratory Data Analysis

**Rented Bike Count**

*Histogram of Rented Bike Count Distribution*

In [ ]:
fig = plt.figure(figsize = (20,5))
sns.set_style('darkgrid')
bins = np.arange(0, 3540, 100).tolist()
data['Rented Bike Count'].hist(bins=bins)
plt.xticks(bins)
plt.xlabel('Rented Bike Count')

**Rented Bike Count vs Season**

In [ ]:
data['Seasons'].unique()

*Box plot of Rented Bike Count vs Seasons*

In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(x=data['Seasons'], y=data['Rented Bike Count'])
plt.show()

**Rented Bike Count vs Holiday**

*Box plot of Rented Bike Count vs Holiday*

In [ ]:
plt.figure(figsize=(4, 6))
sns.boxplot(x=data['Holiday'], y=data['Rented Bike Count'])
plt.show()

**Traffic Volume vs Temperature**

*Plot of Rented Bike Count vs Temperature*

In [ ]:
fig = sns.jointplot(x=data['Temperature(°C)'], y=data['Rented Bike Count'], kind='reg')

**Traffic Volume vs Rainfall**

*Plot of Rented Bike Count vs Rainfall*

In [ ]:
fig = sns.jointplot(x=data['Rainfall(mm)'], y=data['Rented Bike Count'], kind='reg')

*Removing outliers*

In [ ]:
outliers = data[(data['Rainfall(mm)'] >= 20)]
data = data.drop(outliers.index)
data.index = np.arange(1, len(data) + 1)
outliers

In [ ]:
fig = sns.jointplot(x=data['Rainfall(mm)'], y=data['Rented Bike Count'], kind='reg')

**Traffic Volume vs Rainfall**

*Plot of Traffic Volume vs Wind Speed*

In [ ]:
fig = sns.jointplot(data=data, x="Wind speed (m/s)", y="Rented Bike Count", kind='reg')

*Removing outliers*

In [ ]:
outliers = data[(data['Wind speed (m/s)'] >6)]
data = data.drop(outliers.index)
data.index = np.arange(1, len(data) + 1)
outliers

In [ ]:
fig = sns.jointplot(data=data, x="Wind speed (m/s)", y="Rented Bike Count", kind='reg')

**Traffic Volume vs Snowfall**

*Plot of Traffic Volume vs Snowfall*

In [ ]:
fig = plt.figure(figsize = (25,15))
ax1 = fig.add_subplot(2,3,1)
ax1.scatter(data=data, x="Snowfall (cm)", y="Rented Bike Count")

*Distribution only with snowy days*

In [ ]:
data_snowy = data.loc[(data['Snowfall (cm)'] > 0)]
#data_snowy = data.loc[(data['weather_main'] == "Snow")]
data_snowy.index = np.arange(1, len(data_snowy) + 1)
data_snowy

In [ ]:
fig = sns.jointplot(data=data_snowy, x="Snowfall (cm)", y="Rented Bike Count", kind='reg')

**Traffic Volume vs Hour**

*Plot of Traffic Volume vs Hour*

In [ ]:
fig = sns.jointplot(data=data_snowy, x="Hour", y="Rented Bike Count", kind='reg')

**Feature Engineering on Date**

In [ ]:
data[['Day','Month','Year']] = data['Date'].str.extract('(\d+)/(\d+)/(\d+)', expand=True)
data = data.drop(['Date'], axis=1)

**Visualization**

*Bike Count vs Year*

In [ ]:
plt.figure(figsize=(4, 6))
sns.boxplot(x=data['Year'], y=data['Rented Bike Count'])
plt.show()

*Bike Count vs Year*

In [ ]:
plt.figure(figsize=(20, 6))
sns.boxplot(x=data['Month'], y=data['Rented Bike Count'])
plt.show()

*Rented Bike Count vs Hour*

In [ ]:
plt.figure(figsize=(20, 6))
sns.boxplot(x=data['Hour'], y=data['Rented Bike Count'])
plt.show()

## Data Preprocessing

*Preprocess Binary Data*

In [ ]:
data = data.replace(to_replace=['No Holiday', 'Holiday'], value=['0', '1'])
data = data.replace(to_replace=['No', 'Yes'], value=['0', '1'])


*Preprocess Non Ordinal Data*

In [ ]:
one_hot = pd.get_dummies(data["Seasons"])
data = data.drop("Seasons",axis = 1)
data = data.join(one_hot)

In [ ]:
X = data.drop('Rented Bike Count', axis=1)
y = data['Rented Bike Count']

In [ ]:
scaler = StandardScaler()
scaler = scaler.fit(X)
X_scaled = scaler.transform(X)

#split the data in attributes and class as well as training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

## Regression Tasks

*Regression Algorithms from Sklearn*

### Linear Regression

In [ ]:
model = linear_model.LinearRegression().fit(X_train, y_train)

# Make predictions using the testing set
y_pred = model.predict(X_test)

# The coefficients
print('Coefficients: \n', model.coef_)
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(y_test, y_pred))


print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

### KNN Regression

In [ ]:
model = KNeighborsRegressor(n_neighbors=3).fit(X_train, y_train)

# Make predictions using the testing set
y_pred = model.predict(X_test)

# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(y_test, y_pred))


print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

*Our Regression Algorithms*

### Linear Regression Function

In [ ]:
print('==================================================')

alpha = [500, 500, 500, 500]
alphaMethod = 'const'
mu = 1
weights, score = linearRegression.linearRegression(X, y, alpha = alpha, mu = mu, convergenceCriterion = 1e-9, 
                                  lossFunction = 'MSE', alphaMethod = alphaMethod, printOutput = True)
yPred = predictLinearRegression(X, weights)
#print('yPred = ', yPred)
print('weights = ', weights)
print('score = ', score)

print('==================================================')

In [ ]:
stop

In [ ]:
data.type()

### KNN

In [ ]:
df = data.astype(float)

In [ ]:
df

In [ ]:
mode = 1 # 1 = KNeighbors; 2 = RadiusNeighbors
n_neighbours = 5
distance_function = 1 # 1 = Euclidean Distance; 2 = Manhattan Distance
radius = 0 # 0 indicates no radius
label = 'Rented Bike Count'
features = ['Temperature(�C)']
target = {'Rented Bike Count': 254.0,\
  'Hour': 0.0,\
  'Temperature(�C)': -5.2,\
  'Humidity(%)': 37.0,\
  'Wind speed (m/s)': 2.2,\
  'Visibility (10m)': 2000.0,\
  'Dew point temperature(�C)': -17.6,\
  'Solar Radiation (MJ/m2)': 0.0,\
  'Rainfall(mm)': 0.0,\
  'Snowfall (cm)': 0.0,\
  'Holiday': 0.0,\
  'Functioning Day': 1.0,\
  'Day': 1.0,\
  'Month': 12.0,\
  'Year': 2017.0,\
  'Autumn': 0.0,\
  'Spring': 0.0,\
  'Summer': 0.0,\
  'Winter': 1.0}

In [ ]:
dictionary = df.to_dict('records')

In [ ]:
dictionary

In [ ]:
knn = KNN(dictionary, label, features, target, mode, n_neighbours, distance_function, radius)
results = knn.run()
print(results)